# **Bigmart Exploratory Data Analysis and Sales Prediction**

Introduction
The study invloves EDA and Sales prediction from Bigmart Data. The study will involve:


1.   Getting the Data
2.   Exploring the Data

1.   Data Processing
2.   Exploring Models and Selecting the Best

## **Getting the Data**

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

### Importing Necessary Files

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
#df = pd.read_csv('/content/drive/MyDrive/MLData/bigmart_data.csv')
#df_test = pd.read_csv('/content/drive/MyDrive/MLData/test.csv')

# df = pd.read_csv('bigmart_data.csv')
# df_test = pd.read_csv('test.csv')

df = pd.read_csv('../input/bigmart-sales-data/Train.csv')
df_test = pd.read_csv('../input/bigmart-sales-data/Test.csv')

df.shape, df_test.shape

## **Exploring the Data**

In [ ]:
df.info(), df_test.info()

In [ ]:
df.head()

#### Exploration of Numerical Features

In [ ]:
df.hist(bins=50, figsize=(20, 15))

In [ ]:
def check_skew(X, attrib):
  for i in attrib:
    print(i, X[i].skew())

In [ ]:
attribs = ['Item_Weight', 'Item_Visibility', 'Item_MRP', 'Item_Outlet_Sales']

check_skew(df, attrib=attribs)

Therefore only `Item_Outlet_Sales` and `Item_Visibility` will need scaling and *log transformation* will be done on them because their distribution is less normal.

#### Exploration of Categorical Features



1.   **`Item_Fat_Content`**



In [ ]:
sns.catplot(x='Item_Fat_Content', kind='count', data=df)

Low Fat, low fat and LF are the same
Regular and reg are the same
Therefore immediate response is editing of the DataFrame for both the test and train(df)

In [ ]:
df['Item_Fat_Content'].replace({'LF': 'Low Fat', 'reg': 'Regular', 'low fat':'Low Fat'}, inplace=True)
df_test['Item_Fat_Content'].replace({'LF': 'Low Fat', 'reg': 'Regular', 'low fat':'Low Fat'}, inplace=True)
sns.catplot(x='Item_Fat_Content', kind='count', data=df)

  2. **`Item_Type`**

In [ ]:
sns.catplot(data = df, x='Item_Type', kind='count', aspect=5)

There are several types and with ML there is need to reduce the curse of dimensionality by categorising the types e.g perishable and non-perishable

  3. **`Outlet_Identifier`**

In [ ]:
df['Outlet_Identifier'].unique()

In [ ]:
sns.catplot(kind='count', x='Outlet_Identifier', data=df, aspect=2)

Outlets of identifier OUT010 and OUT019 share the almost equal number of items bought. 

  4. **`Outlet_Size`**

In [ ]:
df['Outlet_Size'].unique()

In [ ]:
sns.catplot(kind='count', x='Outlet_Size', data=df)

  5. **`Outlet_Establishment_Year`**

In [ ]:
sns.catplot(data=df, kind='count', x='Outlet_Establishment_Year', aspect=2)

  6. **`Outlet_Type`**

In [ ]:
sns.catplot(data=df, kind='count', x='Outlet_Type', aspect=2)

#### Relationship between Independent Numerical Features and `Item_Outlet_Sales`

  1. **`Item_Weight`**

In [ ]:
sns.scatterplot(x='Item_Weight', y='Item_Outlet_Sales', data=df, alpha=0.3)

  2. **`Item_Visibility`**

In [ ]:
sns.scatterplot(x='Item_Visibility', y='Item_Outlet_Sales', data=df, alpha=0.3)

   3. **`Item_MRP`**

In [ ]:
plt.figure(figsize=(24, 16))
g = sns.scatterplot(x='Item_MRP', y='Item_Outlet_Sales', data=df, alpha=0.3,)
g.set_xticks(np.arange(28, 270, 10))
g.set_xticklabels(np.arange(28, 270, 10), rotation=90)
plt.show()

   **Observations**
1. For `Item_Visibility` it is strange for it to be zero.
2. For `Item_MRP` there are classes in relation to the `Item_Outlet_Sales` which will help us engineer a feature

#### Relationship between Independent Categorical Features and `Item_Outlet_Sales`

   1. **`Item_Fat_Content`**

In [ ]:
sns.catplot(x='Item_Fat_Content', y='Item_Outlet_Sales', data=df, kind='violin')

   2. **`Item_Type`**

In [ ]:
sns.catplot(x='Item_Type', y='Item_Outlet_Sales', kind='violin', data=df, aspect=4)

   3. **`Outlet_Identifier`**

In [ ]:
sns.catplot(x='Outlet_Identifier', y='Item_Outlet_Sales', kind='violin', data=df, aspect=4)

   4. **`Outlet_Size`**

In [ ]:
sns.catplot(x='Outlet_Size', y='Item_Outlet_Sales', kind='violin', data=df)

   5. **`Outlet_Type`**

In [ ]:
sns.catplot(x='Outlet_Type', y='Item_Outlet_Sales', kind='violin', data=df, aspect=2)

   6. **`Outlet_Location_Type`**

In [ ]:
sns.catplot(x='Outlet_Location_Type', y='Item_Outlet_Sales', data=df, kind='violin')

   **Observations**
1. Low Fat and Regular classes of `Item_Fat_Content` look similar and not distinct just like the distribution of `Item_Outlet_Sales` for `Item_Type`
2. OUT010 and OUT019 identifiers are similar in distriution for `Item_Outlet_Sales` but very different from the rest
3. Tier 1 and Tier 2 locations of `Outlet_Location_Type` are similar in distribution for `Item_Outlet_Sales`
4. Grocery Store type has its distribution at lower `Item_Outlet_Sales`

## **Data Processing**

#### Imputing Missing Data
For continuous features the mode, median, and mean can be used but for categorical features the mode or a selected value can be used to impute missing values.

In [ ]:
# Finding columns with missing values
df.isna().sum(), df_test.isna().sum()

   **Observations**
1. Only 2 features (`Item_Weight` and `Outlet_Size`) have missing values and for the `Item_Weight` we will employ the _mean_ to impute the missing values and for the `Outlet_Size` we will employ the _mode_ for both the train and test datasets.

In [ ]:
#Imputing Missing Values in the Item Weight Feature
df['Item_Weight'].fillna(df['Item_Weight'].mean(), inplace=True)
df_test['Item_Weight'].fillna(df_test['Item_Weight'].mean(), inplace=True)

df['Item_Weight'].isna().sum(), df_test['Item_Weight'].isna().sum()

In [ ]:
#Imputing Missing Values in the Outlet_Size Feature

df['Outlet_Size'].fillna('Medium', inplace=True)
df_test['Outlet_Size'].fillna('Medium', inplace=True)

df['Outlet_Size'].isna().sum(), df_test['Outlet_Size'].isna().sum()

#### Replacing Zero Values in the `Item_Visibility` Feature

In [ ]:
a = df[df['Item_Visibility'] != 0.0]['Item_Visibility'].median()

In [ ]:
df['Item_Visibility'] = df['Item_Visibility'].replace(0.00, a)
df['Item_Visibility'].plot(kind='hist', bins=100)

#### Feature Engineering

This will involve creating new features that will offer more value in terms of predicting sales.The new feature will be:
1. Age: Years of operation considering the dataset is dated 2013
2. PpW: Item_MRP/Item_Weight
3. Item_Kind: Perishable or Not_Perishable from Item_Type
4. Item_Cate: From the Item_Identifier
5. IPC(Item_Price_Class): Binned Item_MRP

**1. Age**

In [ ]:
df['Age'] = 2013 - df['Outlet_Establishment_Year']
df_test['Age'] = 2013 - df_test['Outlet_Establishment_Year']
df.head()

**2. PpW**

In [ ]:
df['PpW'] = df['Item_MRP']/df['Item_Weight']
df_test['PpW'] = df_test['Item_MRP']/df['Item_Weight']
df.head()

**3. Item_Kind**

In [ ]:
def kindify(X, feature):
    perishable = ["Breads", "Breakfast", "Dairy", "Fruits and Vegetables", "Meat", "Seafood"]
    non_perishable = ["Baking Goods", "Canned", "Frozen Foods", "Hard Drinks", "Health and Hygiene", 
                      "Household", "Soft Drinks"]
    kind_list = []
    for i in X[feature]:
        if (i in perishable):
            kind_list.append('P')
        elif (i in non_perishable):
            kind_list.append('NP')
        else:
            kind_list.append('NS')
    return kind_list

In [ ]:
x = kindify(df, 'Item_Type')
df['Item_Kind'] = x
t = kindify(df_test, 'Item_Type')
df_test['Item_Kind'] = t
df.head(10)[['Item_Type', 'Item_Kind']]

**4. Item_Cate**

In [ ]:
df['Item_Cate'] = df['Item_Identifier'].replace({'^DR[A-Z]*[0-9]*': 'DR', 
                                                 '^FD[A-Z]*[0-9]*': 'FD', '^NC[A-Z]*[0-9]*': 'NC'}, regex=True)
df_test['Item_Cate'] = df_test['Item_Identifier'].replace({'^DR[A-Z]*[0-9]*': 'DR', 
                                                 '^FD[A-Z]*[0-9]*': 'FD', '^NC[A-Z]*[0-9]*': 'NC'}, regex=True)
df.head()

In [ ]:
df.groupby(['Item_Type'])['Item_Cate'].value_counts().unstack()

**Observations**

From the new Feature `Item_Cate` and crosstabbing it with `Item_Type` we realise Health and Hygiene, Household, Others are not consumable from their categorisation NC. This then will introduce a new class in `Item_Fat_Content` known as Not Edible.

In [ ]:
df['Item_Fat_Content'][df['Item_Cate'] == 'NC'] = 'Not Edible'
df_test['Item_Fat_Content'][df_test['Item_Cate'] == 'NC'] = 'Not Edible'
df['Item_Fat_Content'].unique(), df_test['Item_Fat_Content'].unique()

**5. IPC(Item_Price_Class)**

In [ ]:
def price_classing(X):
    if X<70:
        return '1'
    elif X in range(70, 136):
        return '2'
    elif X in range(136, 203):
        return '3'
    elif X>= 203:
        return '4'

In [ ]:
df['IPC'] = df['Item_MRP'].astype(int).apply(price_classing)
df_test['IPC'] = df_test['Item_MRP'].astype(int).apply(price_classing)

#### Encoding Categorical Features
##### Ordinal Features
1. `Item_Fat_Content`
2. `Outlet_Size`
3. `Outlet_Location_Type`


##### Non-Ordinal Features
1. `Outlet_Type`
2. `Item_Cate`
3. `Item_Kind`

The Ordinal Features will undergo Label Encoding
The Non-ordinal will undergo One Hot Encoding
NB: For common encoding both train and test data have to be concatenated.

In [ ]:
#Label Encoding
from sklearn.preprocessing import LabelEncoder

fatle = LabelEncoder() #Label Encoder for Item_Fat_Content
fatle.fit(df['Item_Fat_Content']) 
sizele = LabelEncoder() #Label Encoder for Outlet_Size
sizele.fit(df['Outlet_Size'])
locle = LabelEncoder() #Label Encoder for Outlet_Location_Type
locle.fit(df['Outlet_Location_Type'])

In [ ]:
df['Item_Fat_Content'] = fatle.transform(df['Item_Fat_Content'])

In [ ]:
df['Outlet_Size'] = sizele.transform(df['Outlet_Size'])

In [ ]:
df['Outlet_Location_Type'] = locle.transform(df['Outlet_Location_Type'])

In [ ]:
# One Hot Encoding
dumm = pd.get_dummies(df[['Outlet_Identifier', 'Outlet_Type', 'Item_Kind', 'Item_Cate']])
dumm.head()

In [ ]:

train = pd.concat([df.drop(['Outlet_Identifier', 'Outlet_Type', 'Item_Kind', 'Item_Cate'], axis=1), dumm], axis=1)
train.columns

#### Checking for Skewness Across all Features and Removal

In [ ]:
num_cols = ['Item_Weight', 'Item_Visibility', 'Item_MRP', 'Item_Outlet_Sales', 'Age', 'PpW', 'IPC']
check_skew(train, num_cols)

**Observation**

`Item_Visibility`, `Item_Outlet_Sales`, `PpW` are marked for removal of skewness considering it is more than 0.5 currently. The removal will be done via log transformation.

In [ ]:
def rem_skew(X, attrib):
    X_ = X.copy()
    for i in attrib:
        X_[i] = np.log10(X_[i])
    return X_

In [ ]:
atr = ['Item_Visibility','Item_Outlet_Sales', 'PpW']
df1 = rem_skew(train, atr)

In [ ]:
check_skew(df1, num_cols)

In [ ]:
df1[['Item_Visibility','Item_Outlet_Sales', 'PpW']]

In [ ]:
corr_matrix=df1.corr()
corr_matrix['Item_Outlet_Sales'].sort_values(ascending=False)

## **Exploring Models and Selecting the Best**
We will explore the following models and select the best to predict the item sales:
1. Ridge
2. Elastic Net
3. Decision Trees
4. Random Forest
5. XGBoost

We will use Root Mean Squared Error to select the best model.

In [ ]:
# First: To split data into train and validation sets

from sklearn.model_selection import train_test_split
X = df1.drop(['Outlet_Establishment_Year', 'Item_Identifier',
              'Item_Outlet_Sales', 'Item_Fat_Content', 'Item_Type', 'Outlet_Size'], axis=1)
target = df1['Item_Outlet_Sales']

X_train, X_valid, target_train, target_valid = train_test_split(X, target, test_size = 0.25, random_state=42)
X_train['IPC'] = X_train['IPC'].astype(int)
X_train.shape,X_valid.shape,target_train.shape,target_valid.shape

**1. Ridge**

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
%time
from sklearn.linear_model import Ridge
rid_reg = Ridge()
rid_reg.fit(X_train, target_train)
predicts_rid = rid_reg.predict(X_train)
rid_mse = mean_squared_error(target_train, predicts_rid)
rid_rmse = np.sqrt(rid_mse)
print(rid_rmse)

**2. Elastic Net**

In [ ]:
%time
from sklearn.linear_model import ElasticNet
en_reg = ElasticNet()
en_reg.fit(X_train, target_train)
predicts_en = en_reg.predict(X_train)
en_mse = mean_squared_error(target_train,predicts_en)
en_rmse = np.sqrt(en_mse)
print(en_rmse)

**3. Decision Tree**

In [ ]:
%time
from sklearn.tree import DecisionTreeRegressor
dct_reg = DecisionTreeRegressor()
dct_reg.fit(X_train, target_train)
predicts_dct = dct_reg.predict(X_train)
dct_mse = mean_squared_error(target_train, predicts_dct)
dct_rmse = np.sqrt(dct_mse)
print(dct_rmse)

**4. Random Forest**

In [ ]:
%time
from sklearn.ensemble import RandomForestRegressor
rf_reg = RandomForestRegressor()
rf_reg.fit(X_train, target_train)
predicts_rf = rf_reg.predict(X_train)
rf_mse = mean_squared_error(target_train, predicts_rf)
rf_rmse = np.sqrt(rf_mse)
print(rf_rmse)

**5. XGBoost**

In [ ]:
%time
from xgboost import XGBRegressor
xgb_reg = XGBRegressor(objective='reg:squarederror')
xgb_reg.fit(X_train, target_train)
predicts_xgb= xgb_reg.predict(X_train)
xgb_mse = mean_squared_error(target_train, predicts_xgb)
xgb_rmse = np.sqrt(xgb_mse)
print(xgb_rmse)

**Observation**
1. With only one fold, all the algorithms have a considerably low RMSE
Therefore, we will do a further test with cross_val_score

In [ ]:
from sklearn.model_selection import cross_val_score

def display_scores(scores):
    print('Scores: ', scores)
    print('Mean Score: ', scores.mean())
    print('Std: ', scores.std())

In [ ]:
%time
scores_rid = cross_val_score(rid_reg, X_train, target_train, scoring = 'neg_mean_squared_error', cv=10)
rid_scores_rmse = np.sqrt(-scores_rid)
display_scores(rid_scores_rmse)

In [ ]:
%time
scores_en = cross_val_score(en_reg, X_train, target_train, scoring = 'neg_mean_squared_error', cv=10)
en_scores_rmse = np.sqrt(-scores_en)
display_scores(en_scores_rmse)

In [ ]:
%time
scores_dct = cross_val_score(dct_reg, X_train, target_train, scoring = 'neg_mean_squared_error', cv=10)
dct_scores_rmse = np.sqrt(-scores_dct)
display_scores(dct_scores_rmse)

In [ ]:
%time
scores_rf = cross_val_score(rf_reg, X_train, target_train, scoring = 'neg_mean_squared_error', cv=10)
rf_scores_rmse = np.sqrt(-scores_rf)
display_scores(rf_scores_rmse)

In [ ]:
%time
scores_xgb = cross_val_score(xgb_reg, X_train, target_train, scoring = 'neg_mean_squared_error', cv=10)
xgb_scores_rmse = np.sqrt(-scores_xgb)
display_scores(xgb_scores_rmse)

**Observation**

The one fold for Random Forest, XGBoost and Decision Tree gave a wrong impression to be performing better. For 10 Folds, there is a change.

From the 5 algorithms, we select Ridge, Random Forest, and XGBoost to perform a GridSearch

#### **HyperTuning**

In [ ]:
from sklearn.model_selection import GridSearchCV

model_params = {
    'Ridge':{
        'model': Ridge(),
        'params':{
            'max_iter': [1000],
            'solver':['lsqr', 'sag']
        }
    },
    'Random_Forest': {
        'model': RandomForestRegressor(),
        'params': {
            'n_estimators': [100, 250],
            'criterion':['mse', 'mae'],
            'max_features': ['sqrt', 'log2'],
            'bootstrap': [False, True]
        }
    },
    'XGBoost':{
        'model': XGBRegressor(),
        'params':{
            'n_estimators': [50, 75, 100],
            'objective': ['reg:squarederror'],
            'booster': ['dart', 'gbtree'],
            'eta': [0.05, 0.1],
        }
    }
}

In [ ]:
%time
scores = []
for model_name, mp in model_params.items():
    rgsr = GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    rgsr.fit(X_train, target_train)
    scores.append({
        'model': model_name,
        'best_score': rgsr.best_score_,
        'best_params': rgsr.best_params_
    })

In [ ]:
results = pd.DataFrame(scores, columns = ['model', 'best_score', 'best_params'])
results

#### Evaluating the Model with the Validation Data

In [ ]:
final_model = rgsr.best_estimator_
X_valid['IPC'] = X_valid['IPC'].astype(int)

valid_predictions = final_model.predict(X_valid)

In [ ]:
print(final_model.score(X_valid, target_valid))
print(np.sqrt(mean_squared_error(target_valid, valid_predictions)))

**Observations**

The accuracy of the XGBoost is the best estimator and is consistent in its RMSE. We will use the model to predict *Item_Outlet_Sales* in the test data.

#### Prediction of *Item_Outlet_Sales* in the Test Data Provided

In [ ]:
df_test.head()

In [ ]:
test_df = df_test.copy()
test_df['Item_Fat_Content'] = fatle.transform(test_df['Item_Fat_Content'])
test_df['Outlet_Size'] = sizele.transform(test_df['Outlet_Size'])
test_df['Outlet_Location_Type'] = locle.transform(test_df['Outlet_Location_Type'])

In [ ]:
test_df

In [ ]:
dummy = pd.get_dummies(test_df[['Outlet_Identifier', 'Outlet_Type', 'Item_Kind', 'Item_Cate']])
dummy

In [ ]:
test = pd.concat([test_df.drop(['Item_Identifier', 'Item_Type', 'Outlet_Identifier', 'Outlet_Establishment_Year', 'Outlet_Type', 'Item_Kind', 'Item_Cate', 'Item_Fat_Content', 'Outlet_Size'], axis=1), dummy], axis=1)
test['IPC'] = test['IPC'].astype(int)
test.head()

In [ ]:
num_cols2 = num_cols.copy()
num_cols2.remove('Item_Outlet_Sales')
check_skew(test, num_cols2)

In [ ]:
atr2 =['Item_Visibility', 'PpW']
test1 = rem_skew(test, atr2)

In [ ]:
predicted_test = final_model.predict(test1)

In [ ]:
pred1 = pd.DataFrame(predicted_test, columns=['Item_Outlet_Sales'])
pred1

In [ ]:
def inverse_log10(X, parm):
  X_ = X.copy()
  for i in parm:
    X_[i] = 10 ** X_[i]

  return X_

In [ ]:
final_sales = inverse_log10(pred1, ['Item_Outlet_Sales'])
final_sales.head()